<a href="https://colab.research.google.com/github/marco-finger/DSF-FS22-MarcoFinger/blob/main/CodingTask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Science Fundamentals for DCBP, S. Haug, University of Bern 

# Coding Task 2

- Deadline (upload to Ilias): 2022-05-15 24:00
- Solutions in lecture on 2022-05-18
- Estimated effort: from 4 to 60 hours
- Images (data) on Ilias in the Data folder, the images.zip file
- Main notebook is number 11 for this task, however, also others may help you. You can also benefit a lot from googeling for code examples


## Subtasks (maximum 12.5 points)

- Upload the images to a folder on your google drive. Write a function which retrieves the filenames of the images, contains a loop converting them to png images and saves them. [about 1 hour] [max 3 points] 
- With some filtering and sthresholding, write a function which converts the colored bins into nice white segments on a black background for one image. [about 1 hour][max 3 points]
- Write an algorithm which counts the number of bins. First approach could be a "sliding window" algorithm, i.e a loop sliding over the image and counting regions with substantial white content. There may also be nice segmentation tools out "there" already. You may google. [about 4 hours][max 3 points] 
- Write an algorithm which does the latter task on all images. As the images are quite different, this is a large task which goes beyond the time budget of this exercise, however, you can give it a try. If it works out for a couple of images, that is already good. [from 4 to 50 hours][max 3.5 points]

Work in teams and have fun ! 

### Substask 1

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install pyheif

In [34]:
# https://www.linuxtut.com/en/c2a85dced32c08aca209/

from PIL import Image
import pyheif

def conv(image_path):
    new_name = image_path.replace('.HEIC', '.png')
    heif_file = pyheif.read(image_path)
    data = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
        )
    data.save(new_name, "PNG")
    print(new_name)

/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5391.png


In [36]:
# https://www.geeksforgeeks.org/python-loop-through-files-of-certain-extensions/

import os

path = "/content/drive/MyDrive/DSF-FS22/data/task2/"

for file in os.listdir(path):
  if file.endswith(".HEIC"):
    print(path + file)
    conv(path + file)
print("Done")

/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5379.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5379.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5387.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5387.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5386.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5386.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5385.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5385.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5384.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5384.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5383.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5383.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5382.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5382.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5381.HEIC
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5381.png
/content/drive/MyDrive/DSF-FS22/data/task2/IMG_5393.HEIC
/content/drive/MyDrive/DSF-FS22/data/ta

### Subtask 2